WA school zip codes sourced from https://eds.ospi.k12.wa.us/DirectoryEDS.aspx

In [8]:
# import packages
import pandas as pd
import glob

In [9]:
fafsa = pd.read_csv('/Users/jStrange/Library/CloudStorage/OneDrive-Personal/Data Science/Data/United Way/FAFSA/FAFSA Graduation Data.csv')
# fafsa.head()

In [10]:
kind = pd.read_csv('/Users/jStrange/Library/CloudStorage/OneDrive-Personal/Data Science/Data/United Way/Kindergarten Readiness/Kindergarten.csv')
kind.head()

,Unnamed: 0,schoolyear,County,DistrictName,SchoolName,Percent
0,0,2014-15,Adams,Othello School District,LUTACAGA ELEMENTARY,0.05376
1,1,2014-15,Adams,Othello School District,HIAWATHA ELEMENTARY SCHOOL,0.32558
2,2,2014-15,Adams,Lind School District,LIND ELEMENTARY SCHOOL,0.92857
3,3,2014-15,Asotin,Clarkston School District,PARKWAY ELEMENTARY,0.42553
4,4,2014-15,Asotin,Clarkston School District,GRANTHAM ELEMENTARY,0.24390


In [11]:
alice = pd.read_csv('/Users/jStrange/Library/CloudStorage/OneDrive-Personal/Data Science/Data/United Way/ALICE/ALICE_zip.csv')
alice.head()

,Unnamed: 0,Year,Zip Code,Households,Poverty Households,ALICE Households
0,780,2021,98001,11820,793,2067
1,781,2021,98002,14012,1866,5176
2,782,2021,98003,18988,2565,6380
3,783,2021,98004,18125,838,2250
4,784,2021,98005,8732,691,1177


In [12]:
zip_raw = pd.read_csv('/Users/jStrange/Library/CloudStorage/OneDrive-Personal/Data Science/Data/United Way/Other Data/Washington_School_Directory_20250109.csv')
# zip_raw.head()

In [13]:
zip = zip_raw[['ZipCode', 'City', 'LEAName','SchoolName','GradeCategory']]

zip['ZipCode'] = zip['ZipCode'].str.split('-').str[0]
zip.head()

/var/folders/13/tb353hv16bg0qsrdngc6r2v40000gn/T/ipykernel_19012/1958076388.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zip['ZipCode'] = zip['ZipCode'].str.split('-').str[0]


,ZipCode,City,LEAName,SchoolName,GradeCategory
0,99371,Washtucna,Washtucna School District,Washtucna Elementary/High School,PK-12
1,99105,Benge,Benge School District,Benge Elementary,Elementary School
2,99344,Othello,Othello School District,Lutacaga Elementary,Elementary School
3,99344,Othello,Othello School District,Hiawatha Elementary School,Elementary School
4,99344,Othello,Othello School District,Othello High School,High School


In [ ]:
# Convert school names to lowercase in both dataframes
fafsa['SchoolName_lower'] = fafsa['Name'].str.lower()
zip['SchoolName_lower'] = zip['SchoolName'].str.lower()

fafsa['City_lower'] = fafsa['City'].str.lower()
zip['City_lower'] = zip['City'].str.lower()

fafsa['District_lower'] = fafsa['DistrictName'].str.lower()
zip['District_lower'] = zip['LEAName'].str.lower()

# Merge the dataframes on the lowercase school names
merged_df = pd.merge(fafsa, zip, left_on=['SchoolName_lower','City','District_lower'], right_on=['SchoolName_lower','City','District_lower'], how='inner')

# Drop the temporary lowercase columns
merged_df.drop(columns=['SchoolName_lower','City','District_lower'], inplace=True)

merged_df.head()

In [7]:
fafsa_zip = merged_df[['SchoolYear','As of Date','ZipCode','FinalCohort','Graduate','Submitted','Completed']]
fafsa_zip.head()

NameError: name 'merged_df' is not defined

In [ ]:
fafsa_zip_aggregated = fafsa_zip.groupby(['SchoolYear', 'As of Date', 'ZipCode']).sum().reset_index()
fafsa_zip_aggregated.head()

In [29]:
fafsa_zip_aggregated.to_csv('/Users/jStrange/Library/CloudStorage/OneDrive-Personal/Data Science/Data/United Way/FAFSA/FAFSA_aggregated.csv')

In [ ]:
# Convert county and district names to lowercase in both dataframes
kind['County_lower'] = kind['County'].str.lower()
kind['DistrictName_lower'] = kind['DistrictName'].str.lower()

zip['County_lower'] = zip['City'].str.lower()  # Assuming 'City' in zip corresponds to 'County' in kind
zip['DistrictName_lower'] = zip['LEAName'].str.lower()

# Merge the dataframes on the lowercase county and district names
merged_kind_zip = pd.merge(kind, zip, left_on=['County_lower', 'DistrictName_lower'], right_on=['County_lower', 'DistrictName_lower'], how='inner')

# Drop the temporary lowercase columns
merged_kind_zip.drop(columns=['County_lower', 'DistrictName_lower'], inplace=True)

merged_kind_zip.head()